# Sparse Bayesian Auxiliary Module _(SBAM)_
In this notebook we will try to explore the `PySINDy` package architecture, in order to easily implement our bayesian version of this algorithm with a sparsity inducing prior.

In `PySINDy`, the main object is the `pysindy.SINDy` model object, built as a subclass of the `BaseEstimator` object from `sklearn` in order to ensure compatibility; this serves as a wrapper for all of the component necessary to the algorithm's implementation, that are

- *differentiation* : the submodules tasked with the computation of the time derivatives
- *feature_library* : the submodules tasked with the construction of the "candidate features" of the linear regression
- *optimizers* : the submodules tasked with performing the linear regression

For each of these tasks, several algorithms alternatives are available, therefore several different submodules can be deployed.

We will implement a "custom" `optimizer` module, that will perform Gibbs sampling of our posterior distribution for all the random parameters defined in the model. The object will retain the samples as an attribute, and will communicate their (conditioned on the masking mode) mean as the best coefficients to the rest of the `PySINDy` package.

## The `BaseOptimizer` class

This is the wrapper class for each optimizer algorithm that the package provides; we will build a optimizer module as a subclass of this wrapper. <a href=https://pysindy.readthedocs.io/en/latest/_modules/pysindy/optimizers/base.html#BaseOptimizer>Source code</a> is available on the documentation. 

The `BaseOptimizer` class implements a `.fit()` method that is called by the `SINDy` object, feeding it the target variable (the time derivative) and the features (built with the `feature_library` module). The `.fit()` itself (after eventually some optional data manipulation such as feature normalization) calls a `._reduce()` method that is supposed to be built into the actual submodule used for optimization and should carry the work to find the regression coefficients and their "presence" in the model, storing them in the class' attributes `.coef_` (array of floats shaped like `(n_targets,n_features)`) and  `.ind_` (array of boolean values shaped like `._coef`).

We will consider "active" those coefficients $\beta_j$ whose $z_j$ is more likely to be 1 rather than 0 in the posterior distribution; that is, it is more probable for them to have some non-zero value (the slab) rather than zero (the spike). For these coefficients, we then calculate the mean only on those samples for which $z_j=1$, that is we will try to estimate $E[\beta_j | z_j = 1]$ 



In [22]:
from pysindy.optimizers import BaseOptimizer
import numpy as np
import pandas as pd
from sklearn.utils.validation import check_X_y
from sklearn.linear_model._base import _preprocess_data
from scipy.stats import beta,gamma,multivariate_normal,binom,mode
from alive_progress import alive_bar # package to print the pretty progress bar. can be installed with pip install alive-progress
from time import sleep # sleep during sampling to ease on cpu




def _rescale_data(X, y, sample_weight):
    """Rescale data so as to support sample_weight"""
    n_samples = X.shape[0]
    sample_weight = np.asarray(sample_weight)
    if sample_weight.ndim == 0:
        sample_weight = np.full(n_samples, sample_weight, dtype=sample_weight.dtype)
    sample_weight = np.sqrt(sample_weight)
    sw_matrix = sparse.dia_matrix((sample_weight, 0), shape=(n_samples, n_samples))
    X = safe_sparse_dot(sw_matrix, X)
    y = safe_sparse_dot(sw_matrix, y)
    return X, y


class SBAM (BaseOptimizer):
    """
    Bayesian Regression with a Spike and Slab type prior Optimizer.
    
    Computes the most likely combination of the coefficient vector
    and the masking vector using Bayesian inference to compute the 
    posterior distribution.

    Parameters
    ----------
    fit_intercept : boolean, optional (default False)
        Whether to calculate the intercept for this model. If set to false, no
        intercept will be used in calculations.

    normalize_columns : boolean, optional (default False)
        Normalize the columns of x (the SINDy library terms) before regression
        by dividing by the L2-norm. Note that the 'normalize' option in sklearn
        is deprecated in sklearn versions >= 1.0 and will be removed.

    copy_X : boolean, optional (default True)
        If True, X will be copied; else, it may be overwritten.
    

    max_iter : int, optional (default 5000)
        Maximum iterations of the optimization algorithm, i.e. the length of the Gibbs sampling chain.

    burn_in : int, optional (default 1500)
        Number of samples from the sampling chain to discard.
    
    verbose : boolean, optional (default False)
        enables verbose

    Attributes
    ----------

    coef_ : array, shape (n_features,) or (n_targets,n_features)
        Coefficients vector.

    ind_ : array, shape (n_features,) or (n_targets,n_features)
        Vector of BOOL values indicating whether or not a feature is considered relevant in the sparse representation.

    samples : list of length (n_targets) 
        Dictionaries containing pandas DataFrames of the samples generated by the Gibbs sampling algorithm.

    ----------|    Model hyperparameters

    a1=0.01 
    a2=0.01 :
        sigma2 ~ InvGamma(a1,a1)
    
    a=1. 
    b=1000. :
        Theta ~ Beta(a,b)
    
    s=5. :
        tau2 ~ InvGamma(0.5,s**2/2)

        

        
    """

    def __init__(
        self, 
        max_iter=5000, 
        burn_in=1500,
        normalize_columns=False, 
        fit_intercept=False, 
        initial_guess=None, 
        copy_X=True,
        tol=1e-5, # apparently needed for compatibility?
        alpha=None, # idk
        verbose=False
        ):

        # super() calls a temporary version of the parent class
        # this way we pass the init parameters to the class itself via inheritance
        # without having to rewrite everything
        super().__init__(max_iter, normalize_columns, fit_intercept, initial_guess, copy_X)

        self.tol=tol
        self.alpha=alpha
        self.burn_in = burn_in
        if self.max_iter <= 0:
            raise ValueError("Max iteration must be > 0")
        self.verbose=verbose

        # ---------------- PRIOR PARAMETERS

        self.a1=0.01
        self.a2=0.01
        self.theta=0.5
        self.a=1.
        self.b=1000.
        self.s=5.
     

    # WE WILL OVERRIDE THE FIT METHOD FROM BaseEstimator SINCE WE WANT DIFFERENT .ind_ attributes

    def fit(self, x_, y, sample_weight=None, **reduce_kws):
        """
        Fit the model.

        Parameters
        ----------
        x_ : array-like, shape (n_samples, n_features)
            Training data

        y : array-like, shape (n_samples,) or (n_samples, n_targets)
            Target values

        sample_weight : float or numpy array of shape (n_samples,), optional
            Individual weights for each sample

        reduce_kws : dict
            Optional keyword arguments to pass to the _reduce method
            (implemented by subclasses)

        Returns
        -------
        self : returns an instance of self
        """

        # ----------- rescaling part
        x_, y = check_X_y(x_, y, accept_sparse=[], y_numeric=True, multi_output=True)

        x, y, X_offset, y_offset, X_scale = _preprocess_data(
            x_,
            y,
            fit_intercept=self.fit_intercept,
            copy=self.copy_X,
            sample_weight=sample_weight,
        )

        if sample_weight is not None:
            x, y = _rescale_data(x, y, sample_weight)

        # self.iters = 0


        # ------------ preparing dimensions, if there is only one target (only one time derivative) then we set it (-1,1) shape
        if y.ndim == 1:
            y = y.reshape(-1, 1)
        coef_shape = (y.shape[1], x.shape[1])
        self.ind_ = np.ones(coef_shape, dtype=bool)

        # ----------- normalization
        self.Theta_ = x # saving original theta
        x_normed = np.copy(x)
        if self.normalize_columns:
            reg = 1 / np.linalg.norm(x, 2, axis=0)
            x_normed = x * reg


        # WHERE THE MAGIC HAPPENS

        self._reduce(x_normed, y, **reduce_kws)
        #self.ind_ = np.abs(self.coef_) > 1e-14 # WE WILL SET THIS IN THE REDUCE METHOD, its gonna be the most probable z vector

        self.history_ = [self.coef_]

        # Rescale coefficients to original units
        if self.normalize_columns:
            self.coef_ = np.multiply(reg, self.coef_)
            if hasattr(self, "coef_full_"):
                self.coef_full_ = np.multiply(reg, self.coef_full_)
            for i in range(np.shape(self.history_)[0]):
                self.history_[i] = np.multiply(reg, self.history_[i])

        self._set_intercept(X_offset, y_offset, X_scale)
        return self
        

    def sampling(self,y,X):


        n_samples = X.shape[0]
        n_features = X.shape[1]

        res = np.empty((self.max_iter,2*n_features + 3))

        # # initialize the beta as least square regression
        res[0,:n_features] = np.linalg.lstsq(X,y,rcond=None)[0]
        # initialize the masking as zeros
        res[0,n_features:2*n_features] = np.zeros(n_features)
        # res["z"][0] = (abs(res['beta'][0]) > 1e-3).astype(int) # try to initialize the masking as those that are higher than a certain threshold 
        # # initialize the sigma as the variance of the residuals
        res[0,-3] = np.var(y - X @ res[0,:n_features])
        # # initialize the tau2 as one and the theta as 0.5
        res[0,-2] = 1.
        res[0,-1] = 0.5

        # compute only once
        XtX = X.T @ X
        Xty = X.T @ y

        # ----------------- BEGIN SAMPLING
        with alive_bar(self.max_iter-1,force_tty=True,bar='filling') as bar:

            for i in range(1,self.max_iter):


                # lets retrieve the previous values for easier coding
                beta_prev = res[i-1,:n_features]
                z_prev = res[i-1,n_features:2*n_features]
                sigma2_prev = res[i-1,-3]
                tau2_prev = res[i-1,-2]
                theta_prev = res[i-1,-1]

                # ------------------ LETS GO WITH THE CONDITIONALS

                # sample theta from a Beta distribution
                theta_new = beta.rvs(self.a + np.sum(z_prev),self.b+np.sum(1-z_prev))

                # sample sigma2 from an inverse gamma
                err = y - X @ beta_prev
                scale = 1./(self.a2 + (err.T @ err)/2)

                # sigma2_new = sigma2_prev
                sigma2_new = 1./gamma.rvs(self.a1+n_samples/2,scale=scale)

                # sample tau2 from an inverse gamma
                scale = 1./((self.s**2)/2 + (beta_prev.T @ beta_prev)/(2*sigma2_new))

                # try to fix tau2!
                # tau2_new = tau2_prev
                tau2_new = 1./gamma.rvs(0.5+0.5*np.sum(z_prev),scale=scale)

                # sample new beta from a multivariate gaussian
                covariance = np.linalg.inv(XtX/sigma2_new + np.eye(n_features)/(sigma2_new*tau2_new))
                mean = covariance @ Xty /sigma2_new # is this right?
                beta_new = multivariate_normal.rvs(mean = mean,cov=covariance)

                # now we sample the zjs
                # in random order
                for j in np.random.permutation(n_features):
                    
                    # grab the current vector
                    z0 = z_prev
                    # set j to zero
                    z0[j] = 0.
                    # get the beta_{-j}
                    bz0 = beta_new * z0

                    # compute the u variables (one for each sample)
                    xj = X[:,j] # the jth feature of each sample
                    u = y - X @ bz0 
                    cond_var = np.sum(xj**2) + 1./tau2_new

                    # compute the chance parameter:
                    # the probability of extracting zj = 0 is prop to (1-theta)
                    # while of extracting zj=1 is (.....) mess 
                    # computing the logarithm of these (l0 and l1) means that the probability of extracting zj=1 is
                    # xi = exp(l1)/(exp(l1)+exp(l0))
                    # we can also write this as
                    # xi = 1/(1+ exp(l0-l1))
                    # this way we can check if exp(l0-l1) overflows and just call it xi = 0

                    l0 = np.log(1-theta_new)
                    l1 = np.log(theta_new) \
                        - 0.5 * np.log(tau2_new*sigma2_new) \
                        + (np.sum(xj*u)**2)/(2*sigma2_new*cond_var) \
                        + 0.5*np.log(sigma2_new/cond_var)

                    el0_l1 = np.exp(l0-l1)
                    if np.isinf(el0_l1):
                        xi = 0
                    else:
                        xi = 1/(1+el0_l1)
                    
                    # extract the zj
                    z_prev[j]=binom.rvs(1,xi)

                # once we extracted all zj, store them:
                z_new = z_prev

                # update everything

                # res[i,"beta"] = beta_new
                res[i,:n_features] = beta_new*z_new
                res[i,n_features:2*n_features] = z_new
                res[i,-3] = sigma2_new
                res[i,-2] = tau2_new
                res[i,-1] = theta_new


                if self.verbose:
                    bar()

            # ---------- END SAMPLING
        
        return res[self.burn_in:,:]


    def _reduce(self,x,y,**sampling_kws):
        """
        Reduce method to actually perform the minimization.
        This method performs a Gibbs sampling of the joint probability distribution
        Under the spike and slab prior method, and will return the coefficients
        as the most probable one, given the most probable masking coefficient.
        """

        n_samples, n_features = x.shape
        n_targets = y.shape[1]
        coef = np.zeros((n_targets,n_features))
        ind = np.zeros((n_targets,n_features))


        # hierarchical columns
        columns = []

        for i in range(n_features):
            columns.append(('beta',str(i)))
            columns.append(('z',str(i)))

        columns.sort()

        columns.append(('sigma2',))
        columns.append(('tau2',))
        columns.append(('theta',))

        index = pd.MultiIndex.from_tuples(columns)

        self.samples = []

        for i in range(n_targets):

            if self.verbose: 
                print("Sampling for target n# {}/{}...".format(i,n_targets-1))

            # we can call y[:,i] because it's been reshaped to (-1,1) even if n_targets=1
            self.samples.append(pd.DataFrame(self.sampling(y[:,i],x,**sampling_kws),columns=index))

            # mask the betas via the z value and calculate the mean
            coef[i] = self.samples[i]['beta'][self.samples[i]['z']==1].mean()
            ind[i] = self.samples[i]['z'].mode().values.astype(bool)
            coef[i] = coef[i]*ind[i].astype(int)

        self.coef_ = coef
        self.ind_ = ind